In [2]:
import json
json_data = open('/home/mattiapdo/Desktop/ADM - Algorithmic Methods/Homework_3/reduced_dblp.json').read()

In [3]:
data = json.loads(json_data)

data is a list of dictionaries... let's see the first item

In [4]:
data[0]

{'authors': [{'author': 'h&eacute;lio marcos paz de almeida',
   'author_id': 255206},
  {'author': 'dorgival olavo guedes neto', 'author_id': 255207},
  {'author': 'wagner meira jr.', 'author_id': 16585},
  {'author': 'mohammed j. zaki', 'author_id': 255208}],
 'id_conference': 'conf/pkdd/2011-1',
 'id_conference_int': 3052,
 'id_publication': 'conf/pkdd/almeidanmz11',
 'id_publication_int': 161535,
 'title': 'is there a best quality metric for graph clusters?'}

so... for each publication we are interested in
- the title (?)                      --> 
                                             data[i]['title']
- the conference                     -->
                                             data[i]['id_conference_int']
- the id_publication_int             --> 
                                             data[i]['id_publication_int']
- the dictionary of the authors      --> 
                                         data[i]['authors'][j]['author'] or
                                         data[i]['authors'][j]['author_id']  ... for j in [0 , num of authors]

We want to create a graph of all the authors... so we can scroll the list and, for each element of the list, check whether each author is in a dictionary 'of all the authors', that is something like:

    {'author': 'pippo' , 'publications': [first_pippo's_publication_id, second_pippo's_publication_id, ...],
      'author': 'mickey' , 'publications': [first_mickey's_publication_id, second_mickey's_publication_id, ...], ... } 

If NO
- add the author to the dictionary, and it's corresponding value: the id of the current publication
- create a new vertex in the graph, named as the id of the author

If YES
- insert the current publication id to the author in the dictionary

Before moving to next item of the list (to another publication), let's add a link between the authors that collaborated for that work (if they are not linked yet)

In [ ]:
#code
import networkx as nx
import matplotlib as plt


Authors = nx.Graph()

In [15]:
def nodes_connected(G, u, v):
    return u in G.neighbors(v)

In [19]:
inverted_dictionary = {}
for dictionary in data:
    # add authors to the graph if they are not in and
    # build the inverted dictionary
    for author in dictionary['authors']:
        if author['author_id'] not in inverted_dictionary:
            Authors.add_node(author['author_id'])
            inverted_dictionary[author['author_id']] = {'name': author['author'], 'publications':[(dictionary['id_publication_int'],dictionary['title'])]}
        else:
            inverted_dictionary[author['author_id']]['publications'].append((dictionary['id_publication_int'],dictionary['title']))
    
    # add edges between authors that appear as contributors in the publication
    # if they are not still connected
    for author1 in dictionary['authors']:
        for author2 in dictionary['authors']:
            if not nodes_connected(Authors, author1['author_id'], author2['author_id']):
                Authors.add_edge(author1['author_id'],author2['author_id'])

Now,
    
    for each couple of authors:
        if they are linked:
            evaluate the Jaccard distance
            set it as weigth for the edge
        

In [33]:
#code
def jaccard_dist (id_1, id_2, inv_dict):
    publ_1 = set([inverted_dictionary[id_1]['publications'][i][0] for i in range(len(inverted_dictionary[id_1]['publications']))])
    publ_2 = set([inverted_dictionary[id_2]['publications'][i][0] for i in range(len(inverted_dictionary[id_2]['publications']))])
    union  = publ_1.union(publ_2)
    intersection = publ_1.intersection(publ_2)
    return 1-len(intersection)/len(union)

In [34]:
for author1 in inverted_dictionary:
    for author2 in inverted_dictionary:
        if nodes_connected(Authors, author1, author2):
            Authors[author1][author2]['weight'] = jaccard_dist(author1, author2, inverted_dictionary)

KeyboardInterrupt: 

## Given a conference in input, return the subgraph induced by the set of authors who published at the input conference at least once

given the list of nodes of the subgraph 
              
    nbunch
    
we can make the induced subgraph as follows:

    Graph.subgraph(nbunch)
    
Return the subgraph induced on nodes in nbunch. The induced subgraph of the graph conOtains the nodes in nbunch and the edges between those nodes.. For **example**

    >>> G = nx.Graph()   # or DiGraph, MultiGraph, MultiDiGraph, etc
    >>> G.add_path([0,1,2,3])
    >>> H = G.subgraph([0,1,2])
    >>> H.edges()
    [(0, 1), (1, 2)]